  This notebook shows an example of neural networks applied to time series classification using Tensorflow. We build a 2-layer neural network with softmax activation and use the raw time series as input for our neural network. 

## Imports

In [106]:
import os
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report, f1_score
import feature_extraction_functions as fef
import matplotlib.pylab as plt
from sklearn import preprocessing
%matplotlib

Using matplotlib backend: MacOSX


## Loading data

The data set we use is x-axis acceleration of people performing 6 different activities (walking (1), walking upstairs (2), walking downstairs (3), sitting (4), standing (5) laying (6)). More details about the data set [here](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones).

In [107]:
os.chdir('../data')
# Raw signals
# X axis
X_train = np.loadtxt('X_train.txt')
X_test = np.loadtxt('X_test.txt')

print("X_train shape : {}".format(X_train.shape))
print("X_test shape : {}".format(X_test.shape))

X_train shape : (7352, 128)
X_test shape : (2947, 128)


In [108]:
# Label vectors
y_train = np.loadtxt('y_train.txt') - 1
y_train = (np.arange(np.unique(y_train).shape[0]) == y_train[:, None]).astype(int)

y_test = np.loadtxt('y_test.txt') - 1
y_test = (np.arange(np.unique(y_test).shape[0]) == y_test[:, None]).astype(int)

print("y_train shape : {}".format(y_train.shape))
print("y_test shape : {}".format(y_test.shape))

label_names = ['Walking', 'Walking upstairs', 'Walking downstairs', 'Sitting', 'Standing', 'Laying']

y_train shape : (7352, 6)
y_test shape : (2947, 6)


 Storing number of features and number of labels for later use.

In [109]:
n_features = X_train.shape[1]
n_labels = y_train.shape[1]

## Network and training parameters

In [152]:
learning_rate = 0.05
training_epoch = 1500
batch_size = 100
display_step = 100
num_layer_1 = n_features
num_layer_2 = n_features #(n_features + n_labels) // 2
l = 0.0
prob = 1.

## Building the Tensorflow graph

  Neural Network input and output

In [153]:
x = tf.placeholder(tf.float32, shape=[None, n_features])
y = tf.placeholder(tf.float32, shape=[None, n_labels])

 Here we build the actual network with two layers: the first layer a n_features number of neurons with softmax activation and a second layer with num_layer_2 neurons and no activation function since we are going to apply softmax to the output of this second layer when defining the neural network cost.

In [154]:
# 1st hidden layer
W_1 = tf.Variable(tf.random_normal([num_layer_1, num_layer_2], stddev=0.01))
b_1 = tf.Variable(tf.zeros([num_layer_2]))
y_1 = tf.nn.softmax(tf.matmul(x, W_1) + b_1)

keep_prob = tf.placeholder(tf.float32)
y_pred = tf.nn.dropout(y_pred, keep_prob)

# Output layer
W_2 = tf.Variable(tf.random_normal([num_layer_2, n_labels], stddev=0.01))
b_2 = tf.Variable(tf.zeros([n_labels]))
y_pred = tf.matmul(y_1, W_2) + b_2

 Cost and optimization:

In [155]:
lam = tf.placeholder(tf.float32)

# Cost function with L2 regularization
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_pred) + 
                      lam * (tf.nn.l2_loss(W_1) + tf.nn.l2_loss(W_2)))
# optimizer
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cost)
# prediction
prediction = tf.nn.softmax(y_pred)

## Training

In [ ]:
sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)

cost_log = []

print("Paramters:")
print("learning rate = {}".format(learning_rate))
print("training_epoch = {}".format(training_epoch))
print("batch_size = {}".format(batch_size))
print("-- Network structure -- ")
print("num_layer_1 = {}".format(num_layer_1))
print("num_layer_2 = {}".format(num_layer_2))
print("-- Regularization --")
print("drop out prob = {}".format(prob))
print("L2 regularization = {}".format(l))
print("\n")

for epoch in range(training_epoch):
    avg_cost = 0.
    num_batch = X_train.shape[0] // batch_size
    
    # We first shuffle the training data
    shuffle = np.random.permutation(X_train.shape[0])
    X_train_s = X_train[shuffle, :]
    y_train_s = y_train[shuffle, :]
    
    for i in range(num_batch):
        batch_x = X_train_s[int(i*batch_size):int((i+1)*batch_size), :]
        batch_y = y_train_s[int(i*batch_size):int((i+1)*batch_size), :]
        train = sess.run(train_step, feed_dict={x: batch_x, y: batch_y, keep_prob: prob, lam: l})
        c =  sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1., lam: 0.})        
        avg_cost += c/num_batch
        
    # Train and cv cost log
    if epoch % display_step == 0:
        print("Epoch n° {:4d}. cost = {:.8f}.".format(epoch, avg_cost))
        
    cost_log.append(avg_cost)
    

print("Training over...")
print("\n")

plt.figure(1)
plt.plot(range(len(cost_log)), cost_log)
plt.xlabel("epoch")
plt.ylabel("cost")
plt.title("Training set cost")


y_train_pred = sess.run(tf.argmax(prediction,1), feed_dict={x: X_train, keep_prob: 1.})
print('Training set report')
print(classification_report(np.argmax(y_train, axis=1), y_train_pred, target_names=label_names))
plt.figure(3)
fef.plot_confusion_matrix(confusion_matrix(np.argmax(y_train, axis=1), y_train_pred), label_names)

y_test_pred = sess.run(tf.argmax(prediction,1), feed_dict={x: X_test, keep_prob: 1.})
print('Test set report')
print(classification_report(np.argmax(y_test, axis=1), y_test_pred, target_names=label_names))
plt.figure(4)
fef.plot_confusion_matrix(confusion_matrix(np.argmax(y_test, axis=1), y_test_pred), label_names)

plt.show()

Paramters:
learning rate = 0.05
training_epoch = 1500
batch_size = 100
-- Network structure -- 
num_layer_1 = 128
num_layer_2 = 128
-- Regularization --
drop out prob = 1.0
L2 regularization = 0.0


Epoch n°    0. cost = 1.30194486.
Epoch n°  100. cost = 0.65251704.


In [ ]:
Paramters:
learning rate = 0.05
training_epoch = 6000
batch_size = 100
-- Network structure -- 
num_layer_1 = 128
num_layer_2 = 128
-- Regularization --
drop out prob = 1.0
L2 regularization = 0.0


Epoch n°    0. cost = 1.29358814.
Epoch n°  100. cost = 0.62734286.
Epoch n°  200. cost = 0.59615321.
Epoch n°  300. cost = 0.55617541.
Epoch n°  400. cost = 0.55240083.
Epoch n°  500. cost = 0.51640089.
Epoch n°  600. cost = 0.52212010.
Epoch n°  700. cost = 0.47001223.
Epoch n°  800. cost = 0.45092447.
Epoch n°  900. cost = 0.43522266.
Epoch n° 1000. cost = 0.41911411.
Epoch n° 1100. cost = 0.45977154.
Epoch n° 1200. cost = 0.37950618.
Epoch n° 1300. cost = 0.37941976.
Epoch n° 1400. cost = 0.36402601.
Epoch n° 1500. cost = 0.36181774.
Epoch n° 1600. cost = 0.35305573.
Epoch n° 1700. cost = 0.32691913.
Epoch n° 1800. cost = 0.33533966.
Epoch n° 1900. cost = 0.32368060.
Epoch n° 2000. cost = 0.30189006.
Epoch n° 2100. cost = 0.29234168.
Epoch n° 2200. cost = 0.32200553.
Epoch n° 2300. cost = 0.28747340.
Epoch n° 2400. cost = 0.29338322.
Epoch n° 2500. cost = 0.29864604.
Epoch n° 2600. cost = 0.29692236.
Epoch n° 2700. cost = 0.28090668.
Epoch n° 2800. cost = 0.26183263.
Epoch n° 2900. cost = 0.25052612.
Epoch n° 3000. cost = 0.25763807.
Epoch n° 3100. cost = 0.24576370.
Epoch n° 3200. cost = 0.28468481.
Epoch n° 3300. cost = 0.23438251.
Epoch n° 3400. cost = 0.23968151.
Epoch n° 3500. cost = 0.24386988.
Epoch n° 3600. cost = 0.28600749.
Epoch n° 3700. cost = 0.22991039.
Epoch n° 3800. cost = 0.21870220.
Epoch n° 3900. cost = 0.23165407.
Epoch n° 4000. cost = 0.22822775.
Epoch n° 4100. cost = 0.21830588.
Epoch n° 4200. cost = 0.21708936.
Epoch n° 4300. cost = 0.22156327.
Epoch n° 4400. cost = 0.21511520.
Epoch n° 4500. cost = 0.19481707.
Epoch n° 4600. cost = 0.20703194.
Epoch n° 4700. cost = 0.22103408.
Epoch n° 4800. cost = 0.19173198.
Epoch n° 4900. cost = 0.20647653.
Epoch n° 5000. cost = 0.18102213.
Epoch n° 5100. cost = 0.21170738.
Epoch n° 5200. cost = 0.18653505.
Epoch n° 5300. cost = 0.17519726.
Epoch n° 5400. cost = 0.18856200.
Epoch n° 5500. cost = 0.17052807.
Epoch n° 5600. cost = 0.17467913.
Epoch n° 5700. cost = 0.17803476.
Epoch n° 5800. cost = 0.20223067.
Epoch n° 5900. cost = 0.16515660.
Training over...


Training set report
                    precision    recall  f1-score   support

           Walking       0.98      0.94      0.96      1226
  Walking upstairs       0.87      0.97      0.92      1073
Walking downstairs       0.98      0.91      0.94       986
           Sitting       0.94      0.90      0.92      1286
          Standing       0.94      0.94      0.94      1374
            Laying       0.92      0.95      0.94      1407

       avg / total       0.94      0.94      0.94      7352

Test set report
                    precision    recall  f1-score   support

           Walking       0.72      0.69      0.71       496
  Walking upstairs       0.56      0.69      0.62       471
Walking downstairs       0.73      0.60      0.66       420
           Sitting       0.37      0.36      0.36       491
          Standing       0.45      0.43      0.44       532
            Laying       0.54      0.56      0.55       537

       avg / total       0.56      0.55      0.55      2947

In [89]:
Paramters:
learning rate = 0.01
training_epoch = 1500
batch_size = 100
-- Network structure -- 
num_layer_1 = 128
num_layer_2 = 128
-- Regularization --
drop out prob = 0.5
L2 regularization = 0.0


Epoch n°    0. cost = 1.69495740.
Epoch n°  100. cost = 0.72698880.
Epoch n°  200. cost = 0.65853535.
Epoch n°  300. cost = 0.62282811.
Epoch n°  400. cost = 0.60026161.
Epoch n°  500. cost = 0.58805539.
Epoch n°  600. cost = 0.57424411.
Epoch n°  700. cost = 0.56162119.
Epoch n°  800. cost = 0.55529996.
Epoch n°  900. cost = 0.54978269.
Epoch n° 1000. cost = 0.54558178.
Epoch n° 1100. cost = 0.53990666.
Epoch n° 1200. cost = 0.54018043.
Epoch n° 1300. cost = 0.53091570.
Epoch n° 1400. cost = 0.52653392.
Training over...


Training set report
                    precision    recall  f1-score   support

           Walking       1.00      1.00      1.00      1226
  Walking upstairs       1.00      0.97      0.99      1073
Walking downstairs       0.97      1.00      0.99       986
           Sitting       0.42      0.24      0.30      1286
          Standing       0.48      0.79      0.60      1374
            Laying       0.73      0.56      0.63      1407

       avg / total       0.75      0.74      0.73      7352

Test set report
                    precision    recall  f1-score   support

           Walking       0.71      0.88      0.79       496
  Walking upstairs       0.69      0.55      0.61       471
Walking downstairs       0.77      0.65      0.70       420
           Sitting       0.33      0.19      0.24       491
          Standing       0.43      0.69      0.53       532
            Laying       0.70      0.60      0.65       537

       avg / total       0.60      0.59      0.58      2947

SyntaxError: invalid syntax (<ipython-input-89-1ae738681042>, line 1)

In [ ]:
Paramters:
learning rate = 0.1
training_epoch = 1500
batch_size = 100
num_layer_1 = 128
num_layer_2 = 67
drop out prob = 1.0
L2 regularization = 0.01


Epoch n°    0. cost = 1.20719103.
//anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
f1 train = 0.2789
f1 test = 0.3002
Epoch n°  100. cost = 0.75465302.
f1 train = 0.6461
f1 test = 0.5750
Epoch n°  200. cost = 0.66458246.
f1 train = 0.6810
f1 test = 0.5738
Epoch n°  300. cost = 0.59180169.
f1 train = 0.7346
f1 test = 0.5902
Epoch n°  400. cost = 0.51990438.
f1 train = 0.7430
f1 test = 0.5763
Epoch n°  500. cost = 0.47497643.
f1 train = 0.7798
f1 test = 0.5744
Epoch n°  600. cost = 0.43417974.
f1 train = 0.7902
f1 test = 0.5654
Epoch n°  700. cost = 0.41675655.
f1 train = 0.8056
f1 test = 0.5692
Epoch n°  800. cost = 0.37648067.
f1 train = 0.8355
f1 test = 0.5560
Epoch n°  900. cost = 0.33151524.
f1 train = 0.8514
f1 test = 0.5654
Epoch n° 1000. cost = 0.29555754.
f1 train = 0.8626
f1 test = 0.5596
Epoch n° 1100. cost = 0.27434672.
f1 train = 0.8791
f1 test = 0.5518
Epoch n° 1200. cost = 0.25071624.
f1 train = 0.8818
f1 test = 0.5670
Epoch n° 1300. cost = 0.25381679.
f1 train = 0.8900
f1 test = 0.5621
Epoch n° 1400. cost = 0.27323280.
f1 train = 0.8681
f1 test = 0.5586
Training over...


Training set report
                    precision    recall  f1-score   support

           Walking       0.87      0.96      0.91      1226
  Walking upstairs       0.73      0.83      0.78      1073
Walking downstairs       0.96      0.71      0.82       986
           Sitting       0.98      0.83      0.89      1286
          Standing       0.92      0.95      0.94      1374
            Laying       0.89      0.99      0.94      1407

       avg / total       0.90      0.89      0.89      7352

Test set report
                    precision    recall  f1-score   support

           Walking       0.69      0.82      0.75       496
  Walking upstairs       0.60      0.69      0.64       471
Walking downstairs       0.77      0.51      0.61       420
           Sitting       0.38      0.32      0.35       491
          Standing       0.42      0.40      0.41       532
            Laying       0.51      0.59      0.55       537

       avg / total       0.55      0.55      0.55      2947


In [ ]:
Paramters:
learning rate = 0.1
training_epoch = 500
batch_size = 100
num_layer_1 = 128
num_layer_2 = 128
drop out prob = 1.0
L2 regularization = 0.01


Epoch n°    0. cost = 1.21072758.
cost_t = 0.00014542. cost_cv = 0.00036468.
Epoch n°   50. cost = 0.81807568.
cost_t = 0.00011210. cost_cv = 0.00034152.
Epoch n°  100. cost = 0.72975950.
cost_t = 0.00009868. cost_cv = 0.00040344.
Epoch n°  150. cost = 0.65608785.
cost_t = 0.00009032. cost_cv = 0.00046475.
Epoch n°  200. cost = 0.58742912.
cost_t = 0.00008191. cost_cv = 0.00058560.
Epoch n°  250. cost = 0.49851472.
cost_t = 0.00006760. cost_cv = 0.00077760.
Epoch n°  300. cost = 0.45754595.
cost_t = 0.00006477. cost_cv = 0.00092820.
Epoch n°  350. cost = 0.42982763.
cost_t = 0.00005749. cost_cv = 0.00116620.
Epoch n°  400. cost = 0.40677153.
cost_t = 0.00005422. cost_cv = 0.00124392.
Epoch n°  450. cost = 0.36661577.
cost_t = 0.00005177. cost_cv = 0.00148626.
Training over...
Training set report
                    precision    recall  f1-score   support

           Walking       0.85      0.88      0.86      1226
  Walking upstairs       0.81      0.79      0.80      1073
Walking downstairs       0.82      0.83      0.83       986
           Sitting       0.95      0.68      0.79      1286
          Standing       0.75      0.98      0.85      1374
            Laying       0.94      0.90      0.92      1407

       avg / total       0.86      0.85      0.85      7352

Test set report
                    precision    recall  f1-score   support

           Walking       0.68      0.72      0.70       496
  Walking upstairs       0.62      0.62      0.62       471
Walking downstairs       0.63      0.57      0.59       420
           Sitting       0.32      0.26      0.29       491
          Standing       0.43      0.56      0.49       532
            Laying       0.58      0.53      0.55       537

       avg / total       0.54      0.54      0.54      2947